In [1]:
from sympy import Symbol, solve
import numpy as np
import math

Ser=    [3022,   0,      0,     0,   0]                     # Ps, Vsx, Vsy, Msx, Msy
Uls=    [4048,   0,      0,     0,   0]                     # Pu, Vux, Vuy, Mux, Muy
Pile=   [1300,  300,     200,     0.3,     3,     0.1]      # P, V, T Capacity of Pile, Diameter of Pile, Pile Spacing, Embed depth
Mat=    [24,    400]                                        # f'c, fy
SpW=    [0,       0,       0,       0]                      # Surchage, Conc, Soil, height
w=      SpW[0]+(SpW[1]+SpW[2])/2*SpW[3]                     # Average of Specific Weight
S=      Pile[3]*Pile[4]

c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\HEC\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
Xi=     np.array([
        [0, 0,    0, -S/2, -S*3/4, -S/2,       0, -S*9/10, -S,       0],
        [0, 0, -S/2,  S/2,  S*3/4,  S/2, -S*9/10,  S*9/10,  0, -S*9/10],
        [0, 0,  S/2, -S/2,      0, -S/2,  S*9/10,       0,  S,  S*9/10],
        [0, 0,    0,  S/2, -S*3/4,  S/2,       0, -S*9/10, -S,       0],
        [0, 0,    0,    0,  S*3/4, -S/2, -S*9/10,  S*9/10,  0, -S*9/10],
        [0, 0,    0,    0,      0,  S/2,  S*9/10,       0,  S,  S*9/10],
        [0, 0,    0,    0,      0,    0,       0, -S*9/10, -S,       0],
        [0, 0,    0,    0,      0,    0,       0,  S*9/10,  0, -S*9/10],
        [0, 0,    0,    0,      0,    0,       0,       0,  S,  S*9/10],
        [0, 0,    0,    0,      0,    0,       0,       0,  0,       0],
                 ])


Yi=     np.array([
        [0,  S/2,   S*3/5,  S/2,  S*3/4,  S,    S,    S,  S,  S*3/2],
        [0, -S/2, -S*3/10,  S/2,  S*3/4,  S,  S/2,    S,  0,      S],
        [0,    0, -S*3/10, -S/2,      0,  0,  S/2,  S/2,  S,      S],
        [0,    0,       0, -S/2, -S*3/4,  0,    0,    0,  0,    S/2],
        [0,    0,       0,    0, -S*3/4, -S, -S/2,    0,  0,      0],
        [0,    0,       0,    0,      0, -S, -S*2, -S/2,  0,      0],
        [0,    0,       0,    0,      0,  0,   -S,   -S, -S,   -S/2],
        [0,    0,       0,    0,      0,  0,    0,   -S, -S,     -S],
        [0,    0,       0,    0,      0,  0,    0,    0, -S,     -S],
        [0,    0,       0,    0,      0,  0,    0,    0,  0, -S*3/2],
                ])

Lxy=    np.array([
         [S,   S,     2*S, 2*S, S*5/2, 2*S, 28*S/10, 28*S/10, 3*S, 28*S/10],
         [S, 2*S, 19*S/10, 2*S, 5*S/2, 3*S,     3*S,     3*S, 3*S,     4*S]
                ])

Xi2=    Xi**2
Yi2=    Yi**2

In [3]:
nlat=   math.ceil(max(Ser[1],Ser[2])/Pile[1])
nver=   math.ceil(Ser[0]/Pile[0])
n=      max(nlat,nver,9)

Pxser =    [
           (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[4])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[3])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
           for i in range(n)
           ]

Pyser =    [
           (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[3])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[4])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
           for i in range(n)
           ]

In [4]:
while (np.max(Pxser)>=Pile[0] or (np.min(Pxser)<=0 and abs(np.min(Pxser))>=Pile[2])) and (np.max(Pyser)>=Pile[0] or (np.min(Pyser)<=0 and abs(np.min(Pyser))>=Pile[2])):      # NG 조건
               
        n = n+1                 # NG 조건 시, n을 1씩 늘려서 진행

        Pxser =    [
        (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[4])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[3])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
        for i in range(n)
                   ]
                        
        Pyser =    [
        (Ser[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Ser[3])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Ser[4])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
        for i in range(n)
                   ]

        if n > 9:              # n 개수 9 이상은 다루지 않음
                print('이건 사람 불러야돼')
                break

else :
        n = n
        Pxser = Pxser
        Pyser = Pyser

if np.max(Pxser)<Pile[0] and (np.min(Pxser)>0 or abs(np.min(Pxser))<Pile[2]):                           # X 방향 배치가 OK 일 시
        dir = 1
        print("Pile n = " + str(n))
        print("Pxser = " + str(Pxser))
        print("Ps = " + str(Pile[0]) + " > " + str(np.max(Pxser)))

        if np.min(Pxser)>0:
                print("Ts = " + str(Pile[2]) + " > 0")
        else :
                print("Ts = " + str(Pile[2]) + " > " + str(abs(np.min(Pxser))))

elif np.max(Pyser)<Pile[0] and (np.min(Pyser)>0 or abs(np.min(Pyser))<Pile[2]):                         # Y 방향 배치가 OK 일 시
        dir = 2
        print("Pile n = " + str(n))
        print("Pyser = " + str(Pyser))
        print("Ps = " + str(Pile[0]) + " > " + str(np.max(Pyser)))
        
        if np.min(Pyser)>0:
                print("Ts = " + str(Pile[2]) + " > 0")
        else :
                print("Ts = " + str(Pile[2]) + " > " + str(abs(np.min(Pyser))))      

Pile n = 9
Pxser = [335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777, 335.77777777777777]
Ps = 1300 > 335.77777777777777
Ts = 200 > 0


In [5]:
Pxuls =    [
        (Uls[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Uls[4])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Uls[3])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
        for i in range(n)
           ]

Pyuls =    [
        (Uls[0]+(w*Lxy[0][n-1]*Lxy[1][n-1]))/n+(abs(Uls[3])*Xi[i][n-1])/Xi2.sum(axis=0)[n-1]+(abs(Uls[4])*Yi[i][n-1])/Yi2.sum(axis=0)[n-1]
        for i in range(n)
           ]

if dir == 1:
    P = Pxuls               # 계수하중에 의한 Pile 반력 (X 방향 배치가 OK일 시)

elif dir == 2:
    P = Pyuls               # 계수하중에 의한 Pile 반력 (Y 방향 배치가 OK일 시)

print("Puls = " + str(P))

Puls = [449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777, 449.77777777777777]


In [6]:
def Ceil(x):
    return (x*20-x*20%1+1)/20                                               # 0.05 올림 식 정의
def ΦVc1(x):
    return Pi*vc*2*(Cx+Cy+4*x)*x                                            # 2 Way Shear at d/2 from Column face 식 정의
def ΦVc2(x,y):
    return Pi*vc/2*x*y                                                      # 1 Way Shear at d from Column face 식 정의
def ΦVc3a(x):
    return Pi*vc*math.pi*(Pile[3]+x)*x                                      # 2 Way Shear at d/2 from Pile (Internal) 식 정의 1
def ΦVc3b(x):
    return Pi*vc*(math.pi*(Pile[3]+x)+2*S)*x                                # 2 Way Shear at d/2 from Pile (Internal) 식 정의 2
def ΦVc4a(x):
    return Pi*vc*(math.pi*(Pile[3]+x)/4+math.sqrt(2)*S+(Pile[3]+x))*x       # 2 Way Shear at Corner Pile 식 정의 1
def ΦVc4b(x):
    return Pi*vc*(math.pi*(Pile[3]+x)/4+S)*x                                # 2 Way Shear at Corner Pile 식 정의 2
def ΦVc5(x):
    return Pi*vc/2*(math.sqrt(2)*S+(Pile[3]+x))*x                           # 1 Way Shear at Corner Pile 식 정의

# Pedestal Size Estimation (Square)

Cx=max(Ceil(math.sqrt(Uls[0]/(Mat[0]*1000))),0.5)

Cx = Cx
Cy = Cx

Pi = 0.75
λ = 1
vc = 0.33 * λ * math.sqrt(Mat[0]) * 1000                                    # 2 way shear 기준 vc (1 way는 이 값의 절반 사용)

print (Cx,Cy)


0.5 0.5


In [7]:
if n == 3:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-sum(P)/2,d)))   

    while True : 
        if (d1/2+Cy/2 < abs(Yi[n-1][n-1])) :

            if (ΦVc1(d1)<sum(P)) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :

            if (ΦVc1(d1)<sum(P)-P[0]/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :

            if (ΦVc1(d1)<sum(P)-P[0]) :
                d1 = d1 + 0.05
            else :
                d1 = d1


        elif (d1/2+Cx/2 < Xi[n-1][n-1] and d1/2+Cy/2<(Yi[0][n-1])) :

            if (ΦVc1(d1)<sum(P)) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :

            if (ΦVc1(d1)<sum(P)/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :

            if (ΦVc1(d1)<(P[1]+P[2])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])) :

            if (ΦVc1(d1)<sum(P)-(P[1]+P[2])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :

            if (ΦVc1(d1)<sum(P)-(P[1]+P[2])/4) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :

            if (ΦVc1(d1)<P[0]/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :

                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[n-1][n-1])+Pile[3]/2<=d1/2+Cy/2<=abs(Yi[0][n-1])) :

            if (ΦVc1(d1)<P[0]) :
                d1 = d1 + 0.05
            else :
                d1 = d1


        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :

            if (ΦVc1(d1)<sum(P)-(P[1]+P[2])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (d1/2+Cy/2 < abs(Yi[n-1][n-1]) and ΦVc1(d1)>sum(P)) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 and ΦVc1(d1)>sum(P)-P[0]/2) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>sum(P)-P[0]) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and d1/2+Cy/2<(Yi[0][n-1]) and ΦVc1(d1)>sum(P)) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 and ΦVc1(d1)>sum(P)/2) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>(P[1]+P[2])/2) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1]) and ΦVc1(d1)>sum(P)-(P[1]+P[2])/2) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2 and ΦVc1(d1)>sum(P)-(P[1]+P[2])/4) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 and ΦVc1(d1)>P[0]/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[n-1][n-1])+Pile[3]/2<=d1/2+Cy/2<=abs(Yi[0][n-1]) and ΦVc1(d1)>P[0]) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[n-1][n-1])<=d1/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2 and ΦVc1(d1)>sum(P)-(P[1]+P[2])/2) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]),abs(P[1]+P[2]))/2/(Pi*vc/2*Lxy[0][n-1]))         
    d2b = Ceil(max(abs(P[1]),abs(P[2]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a, d2b)
    
    while True : 

        if (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) : 

            if max(abs(P[0]),abs(P[1]+P[2]))>max(abs(P[1]),abs(P[2])) :
                L = Lxy[0][n-1]

                if abs(P[0])<abs(P[1]+P[2]):
                    if d2+Cy/2 < abs(Yi[n-1][n-1]) and ΦVc2(d2,L)<abs(P[1]+P[2]):
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

                else :
                    if d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L)<abs(P[0]):
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2])):
                        d2 = d2 + 0.05
                else:
                        d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2):

            if max(abs(P[0]),abs(P[1]+P[2])/2)>max(abs(P[1]),abs(P[2])) :
                L = Lxy[0][n-1]

                if abs(P[0])<abs(P[1]+P[2])/2:
                    if (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) and ΦVc2(d2,L)<abs(P[1]+P[2])/2 :
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

                else :
                    if d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L)<abs(P[0]):
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2])):
                        d2 = d2 + 0.05
                else:
                        d2 = d2
        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2):

            if abs(P[0])>max(abs(P[1]),abs(P[2])) :
                L = Lxy[0][n-1]

                if d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L)<abs(P[0]):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :

            if abs(P[0])/2>max(abs(P[1]),abs(P[2])) :
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<abs(P[0])/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2 < d2+Cy/2) :

            L = Lxy[1][n-1]
        
            if d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2])):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) :

            if max(abs(P[0]),abs(P[1]+P[2]))>max(abs(P[1]),abs(P[2]))/2 :
                L = Lxy[0][n-1]

                if abs(P[0])<abs(P[1]+P[2]):
                    if (d2+Cy/2 < abs(Yi[n-1][n-1])) and ΦVc2(d2,L)<abs(P[1]+P[2]) :
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

                else :
                    if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]):
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2]))/2:
                        d2 = d2 + 0.05
                else:
                        d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :

            if max(abs(P[0]),abs(P[1]+P[2])/2)>max(abs(P[1]),abs(P[2]))/2 :
                L = Lxy[0][n-1]

                if abs(P[0])<abs(P[1]+P[2])/2:
                    if (Yi[n-1][n-1]<=d2+Cy/2<=Yi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<abs(P[1]+P[2])/2 :
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

                else :
                    if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]):
                        d2 = d2 + 0.05
                    else:
                        d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2]))/2:
                        d2 = d2 + 0.05
                else:
                        d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2) :

            if abs(P[0])>max(abs(P[1]),abs(P[2]))/2 :
                L = Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]) :
                    d2 = d2 + 0.05
                else:
                    d2 = d2


            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2]))/2:
                        d2 = d2 + 0.05
                else:
                        d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :

            if abs(P[0])/2>max(abs(P[1]),abs(P[2]))/2 :
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<abs(P[0])/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2]))/2:
                        d2 = d2 + 0.05
                else:
                        d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2 < d2+Cy/2) :

            L = Lxy[1][n-1]
        
            if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[1]),abs(P[2]))/2:
                    d2 = d2 + 0.05
            else:
                    d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) :

            L = Lxy[0][n-1]

            if abs(P[0])<abs(P[1]+P[2]):
                if (d2+Cy/2 < abs(Yi[n-1][n-1])) and ΦVc2(d2,L)<abs(P[1]+P[2]) :
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else :
                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]):
                    d2 = d2 + 0.05
                else:
                    d2 = d2


        elif (Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :

            L = Lxy[0][n-1]

            if abs(P[0])<abs(P[1]+P[2])/2:
                if (Yi[n-1][n-1]<=d2+Cy/2<=Yi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<abs(P[1]+P[2])/2 :
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else :
                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2) :

            L = Lxy[0][n-1]

            if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<abs(P[0]) :
                d2 = d2 + 0.05
            else:
                d2 = d2


        elif (Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :

            L = Lxy[0][n-1]

            if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<abs(P[0])/2:
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) : 
            
            L = Lxy[0][n-1]
            d2 = d2

        if ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2])),abs(P[0]),abs(P[1]+P[2]))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2])),abs(P[0]),abs(P[1]+P[2])/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2])),abs(P[0]))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2])),abs(P[0])/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[1]),abs(P[2]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]),abs(P[1]+P[2]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]),abs(P[1]+P[2])/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[1]),abs(P[2]))/2,abs(P[0])/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[1]),abs(P[2]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (d2+Cy/2 < abs(Yi[n-1][n-1])) and ΦVc2(d2,L)>max(abs(P[0]),abs(P[1]+P[2]))) or \
           ((Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=d2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) and ΦVc2(d2,L)>max(abs(P[0]),abs(P[1]+P[2])/2)) or \
           ((Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[0]))) or \
           ((Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(abs(P[0])/2)) or \
           ((Xi[n-1][n-1]+Pile[3]/2 < d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2)) :
             break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[1]),abs(P[1]+P[2]),abs(P[2]+P[0])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4a = Ceil(max(solve(ΦVc4a(d)-abs(P[0]),d)))
    d4b = Ceil(max(solve(ΦVc4b(d)-max(abs(P[1]),abs(P[2])),d)))
    d4 = max(d4a,d4b)

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(abs(P[1]),abs(P[2])),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if dt/2+Cy/2 < abs(Yi[n-1][n-1]) :
         Vu1 = sum(P)
    elif dt/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])<=dt/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 :
         Vu1 = sum(P)-P[0]/2
    elif dt/2+Cx/2 < Xi[n-1][n-1] and abs(Yi[0][n-1])+Pile[3]/2<dt/2+Cy/2 : 
         Vu1 = sum(P)-P[0]
    elif dt/2+Cx/2 < Xi[n-1][n-1] and dt/2+Cy/2<(Yi[0][n-1]) :
         Vu1 = sum(P)
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])<=dt/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 :
         Vu1 = sum(P)/2
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])+Pile[3]/2<dt/2+Cy/2 :
         Vu1 = (P[1]+P[2])/2
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])+Pile[3]/2<=dt/2+Cy/2<=abs(Yi[0][n-1]) : # 1
         Vu1 = sum(P)-(P[1]+P[2])/2
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[n-1][n-1])<=dt/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2 : # 2
         Vu1 = sum(P)-(P[1]+P[2])/4
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and abs(Yi[0][n-1])<=dt/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2 : # 3
         Vu1 = P[0]/2
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<dt/2+Cy/2 : # 4
         Vu1 = 0
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and abs(Yi[n-1][n-1])+Pile[3]/2<=dt/2+Cy/2<=abs(Yi[0][n-1]) : # 5
         Vu1 = P[0]
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and abs(Yi[n-1][n-1])<=dt/2+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2 : # 6
         Vu1 = sum(P)-(P[1]+P[2])/2
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) and (dt+Cy/2 < abs(Yi[n-1][n-1])):
        Vu2 = max(max(abs(P[1]),abs(P[2])),abs(P[0]),abs(P[1]+P[2]))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=dt+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2):
        Vu2 = max(max(abs(P[1]),abs(P[2])),abs(P[0]),abs(P[1]+P[2])/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<dt+Cy/2):
        Vu2 = max(max(abs(P[1]),abs(P[2])),abs(P[0]))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[1]),abs(P[2])),abs(P[0])/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[1]),abs(P[2]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (dt+Cy/2 < abs(Yi[n-1][n-1])) :
        Vu2 = max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]),abs(P[1]+P[2]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=dt+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :
        Vu2 = max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]),abs(P[1]+P[2])/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(max(abs(P[1]),abs(P[2]))/2,abs(P[0]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]) :
        Vu2 = max(max(abs(P[1]),abs(P[2]))/2,abs(P[0])/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(max(abs(P[1]),abs(P[2]))/2)
    elif (Xi[n-1][n-1]+Pile[3]/2 < dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (dt+Cy/2 < abs(Yi[n-1][n-1])) :
        Vu2 = max(abs(P[0]),abs(P[1]+P[2]))
    elif (Xi[n-1][n-1]+Pile[3]/2 < dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])<=dt+Cy/2<=abs(Yi[n-1][n-1])+Pile[3]/2) :
        Vu2 = max(abs(P[0]),abs(P[1]+P[2])/2)
    elif (Xi[n-1][n-1]+Pile[3]/2 < dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])) and (abs(Yi[n-1][n-1])+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]))
    elif (Xi[n-1][n-1]+Pile[3]/2 < dt+Cx/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]) :
        Vu2 = max(abs(P[0])/2)
    elif (Xi[n-1][n-1]+Pile[3]/2 < dt+Cx/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = 0
    else :
        Vu2 = 0

    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[1]),abs(P[1]+P[2]),abs(P[2]+P[0]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산
        
    if d4a>=d4b : 
        Vu4 = abs(P[0])
        ΦVc4 = ΦVc4a(dt)
    else :
        Vu4 = max(abs(P[1]),abs(P[2]))
        ΦVc4 = ΦVc4b(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(P[1],P[2])

    print(round(d1,2),round(d2,2),round(d3,2),round(d4,2),round(d5,2))
    print("d = " + str(round(dt,2)) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")


In [8]:
if n == 4:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-sum(P)/2,d)))           

    while True :

        if (d1/2+Cx/2 < abs(Xi[n-1][n-1])) :
          
            if ΦVc1(d1)<sum(P) : 
                d1 = d1 + 0.05
            else : 
                d1 = d1

        if (d1/2+Cy/2 < abs(Yi[0][n-1])) :
          
            if ΦVc1(d1)<sum(P) : 
                d1 = d1 + 0.05
            else : 
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        
            if ΦVc1(d1)<3/4*sum(P) :
                d1 = d1 + 0.05
            else : 
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
        
            if ΦVc1(d1)<sum(P)/2 :
                d1 = d1 + 0.05
            else : 
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
            
            if ΦVc1(d1)<sum(P)/2 :
                d1 = d1 + 0.05
            else : 
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
            
            d1 = d1
        
        if (d1/2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc1(d1)>sum(P)) or \
           (d1/2+Cy/2 < abs(Yi[0][n-1]) and ΦVc1(d1)>sum(P)) or \
           ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc1(d1)>3/4*sum(P)) or \
           ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) and ΦVc1(d1)>sum(P)/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc1(d1)>sum(P)/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2/(Pi*vc/2*Lxy[0][n-1]))                   
    d2b = Ceil(max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)
    
    while True :

        if (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]+P[1]),abs(P[2]+P[3]))>max(abs(P[0]+P[2]),abs(P[1]+P[3])):
                L = Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2>max(abs(P[0]+P[2]),abs(P[1]+P[3])):
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]
        
            if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            if max(abs(P[0]+P[1]),abs(P[2]+P[3]))>max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2:
                L = Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2            

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2>max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2:
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]
        
            if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2:
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            L = Lxy[0][n-1]

            if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3])):
                d2 = d2 + 0.05
            else:
                d2 = d2
      

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            L = Lxy[0][n-1]

            if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[2]+P[3])):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[0][n-1]
            d2 = d2

        if ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(max(abs(P[0]+P[2]),abs(P[1]+P[3])),max(abs(P[0]+P[1]),abs(P[2]+P[3])))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[0]+P[2]),abs(P[1]+P[3])),max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[0]+P[2]),abs(P[1]+P[3]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2,max(abs(P[0]+P[1]),abs(P[2]+P[3])))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2,max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[2]+P[3]))) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[1]),abs(P[2]+P[3]),abs(P[0]+P[2]),abs(P[1]+P[3])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4 = Ceil(max(solve(ΦVc4b(d)-max(list(map(abs,P))),d)))

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(list(map(abs,P))),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if dt/2+Cx/2 < abs(Xi[n-1][n-1]) :
         Vu1 = sum(P)        
    elif dt/2+Cy/2 < abs(Yi[0][n-1]) :
         Vu1 = sum(P)
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 :
         Vu1 = 3/4*sum(P)
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2 :
         Vu1 = sum(P)/2
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 :
         Vu1 = sum(P)/2
    elif Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 :
         Vu1 = 0
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(max(abs(P[0]+P[2]),abs(P[1]+P[3])),max(abs(P[0]+P[1]),abs(P[2]+P[3])))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2):
        Vu2 = max(max(abs(P[0]+P[2]),abs(P[1]+P[3])),max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2):
        Vu2 = max(abs(P[0]+P[2]),abs(P[1]+P[3]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) :    
        Vu2 = max(max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2,max(abs(P[0]+P[1]),abs(P[2]+P[3])))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2):
        Vu2 = max(max(abs(P[0]+P[2]),abs(P[1]+P[3]))/2,max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2):
        Vu2 = max(abs(P[0]+P[2]),abs(P[1]+P[3]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(abs(P[0]+P[1]),abs(P[2]+P[3]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2):
        Vu2 = max(abs(P[0]+P[1]),abs(P[2]+P[3]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2):
        Vu2 = 0
    else :
        Vu2 = 0

    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[1]),abs(P[2]+P[3]),abs(P[0]+P[2]),abs(P[1]+P[3]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산
        
    Vu4 = max(list(map(abs,P)))
    ΦVc4 = ΦVc4b(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(list(map(abs,P)))

    print(d1,d2,d3,d4,d5)
    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")


In [9]:
if n == 5:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-(sum(P)-P[2])/2,d)))

    while True :

        if d1/2+Cx/2 < abs(Xi[n-1][n-1]) :
            
            if ΦVc1(d1)<sum(P)-P[2] :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif d1/2+Cy/2 < abs(Yi[0][n-1]) : 

            if ΦVc1(d1)<sum(P)-P[2] :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
            
            if ΦVc1(d1)<3/4*(sum(P)-P[2]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2)) :

            if ΦVc1(d1)<(sum(P)-P[2])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1      

        elif ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2)) :
            
            if ΦVc1(d1)<(sum(P)-P[2])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1        

        elif ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2)) :
            
            d1 = d1        
            
        if (d1/2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc1(d1)>sum(P)-P[2]) or \
           (d1/2+Cy/2 < abs(Yi[0][n-1]) and ΦVc1(d1)>sum(P)-P[2]) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>3/4*(sum(P)-P[2])) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>(sum(P)-P[2])/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(sum(P)-P[2])/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2/(Pi*vc/2*Lxy[0][n-1]))                   
    d2b = Ceil(max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)
    
    while True :

        if (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]+P[1]),abs(P[3]+P[4]))>max(abs(P[0]+P[3]),abs(P[1]+P[4])):
                L = Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2>max(abs(P[0]+P[3]),abs(P[1]+P[4])):
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]
        
            if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            if max(abs(P[0]+P[1]),abs(P[3]+P[4]))>max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2:
                L = Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2            

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2>max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2:
                L = Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L = Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]
        
            if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2:
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            L = Lxy[0][n-1]

            if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4])):
                d2 = d2 + 0.05
            else:
                d2 = d2           

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            L = Lxy[0][n-1]

            if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]),abs(P[3]+P[4])):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):
            
            L = Lxy[0][n-1]
            d2 = d2

        if ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(max(abs(P[0]+P[3]),abs(P[1]+P[4])),max(abs(P[0]+P[1]),abs(P[3]+P[4])))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[0]+P[3]),abs(P[1]+P[4])),max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[0]+P[3]),abs(P[1]+P[4]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2,max(abs(P[0]+P[1]),abs(P[3]+P[4])))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2,max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L)>max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[3]+P[4]))) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[2]),abs(P[1]+P[2]),abs(P[3]+P[2]),abs(P[4]+P[2]),abs(P[0]+P[1]),abs(P[0]+P[3]),abs(P[1]+P[4]),abs(P[3]+P[4])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4 = Ceil(max(solve(ΦVc4b(d)-max(P[0],P[1],P[3],P[4]),d)))

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(P[0],P[1],P[3],P[4]),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if dt/2+Cx/2 < abs(Xi[n-1][n-1]) :
         Vu1 = sum(P)-P[2]
    elif dt/2+Cy/2 < abs(Yi[0][n-1]) : 
         Vu1 = sum(P)-P[2]
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 :
         Vu1 = 3/4*(sum(P)-P[2])
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2 :
         Vu1 = (sum(P)-P[2])/2
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 :
         Vu1 = (sum(P)-P[2])/2        
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2 :
         Vu1 = 0 
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(max(abs(P[0]+P[3]),abs(P[1]+P[4])),max(abs(P[0]+P[1]),abs(P[3]+P[4])))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[3]),abs(P[1]+P[4])),max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]),abs(P[1]+P[4]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2,max(abs(P[0]+P[1]),abs(P[3]+P[4])))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2,max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]),abs(P[1]+P[4]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(abs(P[0]+P[1]),abs(P[3]+P[4]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(abs(P[0]+P[1]),abs(P[3]+P[4]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = 0
    else :
        Vu2 = 0

    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[2]),abs(P[1]+P[2]),abs(P[3]+P[2]),abs(P[4]+P[2]),abs(P[0]+P[1]),abs(P[0]+P[3]),abs(P[1]+P[4]),abs(P[3]+P[4]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산
        
    Vu4 = max(P[0],P[1],P[3],P[4])
    ΦVc4 = ΦVc4b(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(P[0],P[1],P[3],P[4])

    print(d1,d2,d3,d4,d5)
    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")

In [10]:
if n == 6:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-(P[0]+P[1]+P[4]+P[5])/2,d)))

    while True :
        
        if (d1/2+Cx/2 < Xi[n-1][n-1]) : 
            
            if ΦVc1(d1)<sum(P) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2 < Yi[0][n-1]) :
        
            if ΦVc1(d1)<(sum(P)-(P[2]+P[3])/2) : 
                d1 = d1 + 0.05 
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2) :
        
            if ΦVc1(d1)<(sum(P)-(P[2]+P[3])/2-(P[0]+P[1]+P[4]+P[5])/2) :
                d1 = d1 + 0.05 
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            
            if ΦVc1(d1)<sum(P)/2 :
                d1 = d1 + 0.05 
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2 < Yi[0][n-1]) : 

            if ΦVc1(d1)<(sum(P)-(P[2]+P[3])) :
                d1 = d1 + 0.05 
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2) :
            
            if ΦVc1(d1)<(sum(P)-(P[2]+P[3])-(P[0]+P[1]+P[4]+P[5])/2) :
                d1 = d1 + 0.05 
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :

            d1 = d1
            
        if (d1/2+Cx/2 < Xi[n-1][n-1] and ΦVc1(d1)>sum(P)) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2 < Yi[0][n-1] and ΦVc1(d1)>(sum(P)-(P[2]+P[3])/2)) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(sum(P)-(P[2]+P[3])/2-(P[0]+P[1]+P[4]+P[5])/2)) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>sum(P)/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2 < Yi[0][n-1] and ΦVc1(d1)>(sum(P)-(P[2]+P[3]))) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(sum(P)-(P[2]+P[3])-(P[0]+P[1]+P[4]+P[5])/2)) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2/(Pi*vc/2*Lxy[0][n-1]))
    d2b = Ceil(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)

    while True :

        if (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]+P[1]),abs(P[4]+P[5])) > max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])) :
                L = Lxy[0][n-1]

                if d2+Cy/2 < Yi[0][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]            

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2 > max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])) :
                L = Lxy[0][n-1]

                if Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]            

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]            

            if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])) :
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]+P[1]),abs(P[4]+P[5])) > max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2 :
                L = Lxy[0][n-1]

                if d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]            

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :

            if max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2 > max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2 :
                L = Lxy[0][n-1]

                if Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]            

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :

            L = Lxy[1][n-1]            

            if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2 :
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])):

            L = Lxy[0][n-1]

            if d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :

            L = Lxy[0][n-1]

            if Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2 :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :

            L = Lxy[1][n-1]
            d2 = d2


        if ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])),max(abs(P[0]+P[1]),abs(P[4]+P[5])))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])),max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L) > max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L) > max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2,max(abs(P[0]+P[1]),abs(P[4]+P[5])))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc2(d2,L) > max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2,max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and Yi[0][n-1]+Pile[3]/2<d2+Cy/2 and ΦVc2(d2,L) > max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2)) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and d2+Cy/2 < abs(Yi[0][n-1]) and ΦVc2(d2,L) > max(abs(P[0]+P[1]),abs(P[4]+P[5]))) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc2(d2,L) > max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[1]),abs(P[2]+P[3]),abs(P[4]+P[5]),abs(P[0]+P[2]),abs(P[2]+P[4]),abs(P[1]+P[3]),abs(P[3]+P[5])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4a = Ceil(max(solve(ΦVc4a(d)-max(P[2],P[3]),d)))
    d4b = Ceil(max(solve(ΦVc4b(d)-max(P[0],P[1],P[4],P[5]),d)))
    d4 = max(d4a,d4b)

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(P[0],P[1],P[4],P[5]),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if dt/2+Cx/2 < Xi[n-1][n-1] :
         Vu1 = sum(P)
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and dt/2+Cy/2 < Yi[0][n-1] :
         Vu1 = sum(P)-(P[2]+P[3])/2
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<=dt/2+Cy/2<Yi[0][n-1]+Pile[3]/2 :
         Vu1 = sum(P)-(P[2]+P[3])/2-(P[0]+P[1]+P[4]+P[5])/2
    elif Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2 :
         Vu1 = sum(P)/2
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and dt/2+Cy/2 < Yi[0][n-1] :
         Vu1 = sum(P)-(P[2]+P[3])
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]<=dt/2+Cy/2<Yi[0][n-1]+Pile[3]/2 :
         Vu1 = sum(P)-(P[2]+P[3])-(P[0]+P[1]+P[4]+P[5])/2
    elif Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2 : 
         Vu1 = 0
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])),max(abs(P[0]+P[1]),abs(P[4]+P[5])))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2):
        Vu2 = max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5])),max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2):
        Vu2 = max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2,max(abs(P[0]+P[1]),abs(P[4]+P[5])))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2,max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[2]+P[4]),abs(P[1]+P[3]+P[5]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(abs(P[0]+P[1]),abs(P[4]+P[5]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(abs(P[0]+P[1]),abs(P[4]+P[5]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = 0
    else :
        Vu2 = 0
        
    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[1]),abs(P[2]+P[3]),abs(P[4]+P[5]),abs(P[0]+P[2]),abs(P[2]+P[4]),abs(P[1]+P[3]),abs(P[3]+P[5]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산

    if d4a>d4b :
        Vu4 = max(P[2],P[3])
        ΦVc4 = ΦVc4a(dt)

    else :
        Vu4 = max(P[0],P[1],P[4],P[5])
        ΦVc4 = ΦVc4b(dt)
        
    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(P[0],P[1],P[4],P[5])

    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")

In [11]:
if n == 7:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-(sum(P)-P[3])/2,d)))

    while True :
        
        if (d1/2+Cy/2 < Yi[1][n-1]) :
           
            if ΦVc1(d1)<(sum(P)-P[3]) :
                d1 = d1 + 0.05
            else :
                d1 = d1
           
        if (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :

            if ΦVc1(d1)<(sum(P)-P[3]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]) : 
            
            if ΦVc1(d1)<(sum(P)-P[3]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(sum(P)-P[3]-(P[0]+P[6])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1
                
        elif (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
            
            if ΦVc1(d1)<(sum(P)-P[3]-(P[0]+P[6])) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(P[0]+P[6]+3/4*(P[1]+P[2]+P[4]+P[5])) :
                d1 = d1 + 0.05
            else :
                d1 = d1
                
        elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(((P[0]+P[6])/2+(P[1]+P[2]+P[4]+P[5])/2)) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
            
            if ΦVc1(d1)<(P[1]+P[2]+P[4]+P[5])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1
                
        elif (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(P[0]+P[6]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<((P[0]+P[6])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
            
            d1 = d1

        if (d1/2+Cy/2 < Yi[1][n-1] and ΦVc1(d1)>(sum(P)-P[3])) or \
           (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2 and ΦVc1(d1)>sum(P)-P[3]) or \
           (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1] and ΦVc1(d1)>sum(P)-P[3]) or \
           (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(sum(P)-P[3]-(P[0]+P[6])/2)) or \
           (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>(sum(P)-P[3]-(P[0]+P[6]))) or \
           (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2 and ΦVc1(d1)>(P[0]+P[6]+3/4*(P[1]+P[2]+P[4]+P[5]))) or \
           (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2)) or \
           (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(((P[0]+P[6])/2+(P[1]+P[2]+P[4]+P[5])/2))) or \
           (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>(P[1]+P[2]+P[4]+P[5])/2) or \
           (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2 and ΦVc1(d1)>P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2) or \
           (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[1][n-1]+Pile[3]/2<=d1/2+Cy/2<=Yi[0][n-1] and ΦVc1(d1)>(P[0]+P[6])) or \
           (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<=d1/2+Cy/2<=Yi[0][n-1]+Pile[3]/2 and ΦVc1(d1)>(P[0]+P[6])/2) or \
           (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            break
            
 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]),abs(P[6]))/2/(Pi*vc/2*Lxy[0][n-1]))
    d2b = Ceil(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)

    while True :

        if (d2+Cx/2 < abs(Xi[n-2][n-1])) and (d2+Cy/2 < abs(Yi[1][n-1])) :

            if max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if d2+Cy/2 < abs(Yi[1][n-1]) and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if d2+Cx/2 < Xi[n-2][n-1] and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2):

            if max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if d2+Cx/2 < Xi[n-2][n-1] and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]),abs(P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (d2+Cx/2 < abs(Xi[n-2][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):

            if max(abs(P[0]),abs(P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]),abs(P[6]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (d2+Cx/2 < abs(Xi[n-2][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1])+Pile[3]/2<dt+Cy/2):

            L = Lxy[1][n-1]             

            if (d2+Cx/2 < abs(Xi[n-2][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[1][n-1])):

            if max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                L = Lxy[0][n-1]

                if d2+Cy/2 < abs(Yi[1][n-1]) and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :

            if max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                L = Lxy[0][n-1]

                if (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]),abs(P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):

            if max(abs(P[0]),abs(P[6])) > max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                L = Lxy[0][n-1]

                if (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]),abs(P[6]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]             

                if (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1])+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]             

            if (Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2 :
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[1][n-1])):

            L = Lxy[0][n-1]

            if d2+Cy/2 < abs(Yi[1][n-1]) and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :

            L = Lxy[0][n-1]

            if (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])):

            L = Lxy[0][n-1]

            if (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) < max(abs(P[1]+P[4]),abs(P[2]+P[5])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):

            L = Lxy[0][n-1]

            if (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]),abs(P[6]))/2 :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (abs(Yi[0][n-1])+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]             
            d2 = d2

        if ((d2+Cx/2 < abs(Xi[n-2][n-1])) and (d2+Cy/2 < abs(Yi[1][n-1])) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])))) or \
           ((d2+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])))) or \
           ((d2+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]),abs(P[6])))) or \
           ((d2+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]),abs(P[6]))/2)) or \
           ((d2+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1])+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L) > max(abs(P[1]+P[4]),abs(P[2]+P[5]))) or \
           ((Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[1][n-1])) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])))) or \
           ((Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])))) or \
           ((Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]),abs(P[6])))) or \
           ((Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]),abs(P[6]))/2)) or \
           ((Xi[n-2][n-1]<=d2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1])+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L) > max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2) or \
           ((Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[1][n-1])) and ΦVc2(d2,L) > max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6]))) or \
           ((Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[1][n-1]<=d2+Cy/2<=Yi[1][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6]))) or \
           ((Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[1][n-1]+Pile[3]/2 < d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(abs(P[0]),abs(P[6]))) or \
           ((Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2) and (abs(Yi[0][n-1]) <= d2+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2) and ΦVc2(d2,L) > max(abs(P[0]),abs(P[6]))/2) or \
           (Xi[n-2][n-1]+Pile[3]/2<d2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[1]),abs(P[0]+P[2]),abs(P[0]+P[3]),abs(P[1]+P[3]),abs(P[1]+P[4]),abs(P[2]+P[3]),abs(P[2]+P[5]),abs(P[3]+P[4]),abs(P[3]+P[5]),abs(P[3]+P[6]),abs(P[4]+P[6]),abs(P[5]+P[6])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4 = Ceil(max(solve(ΦVc4a(d)-max(P[0],P[1],P[2],P[4],P[5],P[6]),d)))

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(P[0],P[6])/(Pi*vc/2*Lxy[0][n-1]))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if (dt/2+Cy/2 < Yi[1][n-1]) :
         Vu1 = sum(P)-P[3]
    elif (dt/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :
         Vu1 = sum(P)-P[3]
    elif (d1/2+Cx/2 < Xi[n-2][n-1] and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]) :
         Vu1 = sum(P)-P[3]
    elif (dt/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
         Vu1 = sum(P)-P[3]-(P[0]+P[6])/2
    elif (dt/2+Cx/2 < Xi[n-2][n-1] and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2) :
         Vu1 = sum(P)-P[3]-(P[0]+P[6])
    elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]<=d1/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :
         Vu1 = (P[0]+P[6]+3/4*(P[1]+P[2]+P[4]+P[5]))
    elif (Xi[n-2][n-1]<=d1/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[1][n-1]+Pile[3]/2<d1/2+Cy/2<Yi[0][n-1]+Pile[3]/2) :
         Vu1 = (P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2)
    elif (Xi[n-2][n-1]<=dt/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
         Vu1 = (((P[0]+P[6])/2+(P[1]+P[2]+P[4]+P[5])/2))
    elif (Xi[n-2][n-1]<=dt/2+Cx/2<=Xi[n-2][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2) :
         Vu1 = (P[1]+P[2]+P[4]+P[5])/2
    elif (Xi[n-2][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[1][n-1]<=dt/2+Cy/2<=Yi[1][n-1]+Pile[3]/2) :
         Vu1 = P[0]+P[6]+(P[1]+P[2]+P[4]+P[5])/2
    elif (Xi[n-2][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[1][n-1]+Pile[3]/2<=dt/2+Cy/2<=Yi[0][n-1]) :
         Vu1 = (P[0]+P[6])
    elif (Xi[n-2][n-1]+Pile[3]/2<dt/2+Cx/2 and Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
         Vu1 = (P[0]+P[6])/2
    elif (Xi[n-2][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
         Vu1 = 0
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-2][n-1])) and (dt+Cy/2 < abs(Yi[1][n-1])):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])))
    elif (dt+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]<=dt+Cy/2<=Yi[1][n-1]+Pile[3]/2):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])))
    elif (dt+Cx/2 < abs(Xi[n-2][n-1])) and (Yi[1][n-1]+Pile[3]/2 < dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]),abs(P[6])))
    elif (dt+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1]) <= dt+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5])),max(abs(P[0]),abs(P[6]))/2)
    elif (dt+Cx/2 < abs(Xi[n-2][n-1])) and (abs(Yi[0][n-1])+Pile[3]/2<dt+Cy/2):
        Vu2 = max(abs(P[1]+P[4]),abs(P[2]+P[5]))
    elif (Xi[n-2][n-1]<=dt+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[1][n-1])):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6])))
    elif (Xi[n-2][n-1]<=dt+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]<=dt+Cy/2<=Yi[1][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6])))
    elif (Xi[n-2][n-1]<=dt+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (Yi[1][n-1]+Pile[3]/2 < dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]),abs(P[6])))
    elif (Xi[n-2][n-1]<=dt+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1]) <= dt+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):
        Vu2 = max(max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2,max(abs(P[0]),abs(P[6]))/2) 
    elif (Xi[n-2][n-1]<=dt+Cx/2<=Xi[n-2][n-1]+Pile[3]/2) and (abs(Yi[0][n-1])+Pile[3]/2<dt+Cy/2):
        Vu2 = max(abs(P[1]+P[4]),abs(P[2]+P[5]))/2
    elif (Xi[n-2][n-1]+Pile[3]/2<dt+Cx/2) and (dt+Cy/2 < abs(Yi[1][n-1])):
        Vu2 = max(abs(P[0]+P[1]+P[2]),abs(P[4]+P[5]+P[6]))
    elif (Xi[n-2][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[1][n-1]<=dt+Cy/2<=Yi[1][n-1]+Pile[3]/2) :
        Vu2 = max(abs(P[0]+(P[1]+P[2])/2),abs((P[4]+P[5])/2+P[6]))
    elif (Xi[n-2][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[1][n-1]+Pile[3]/2 < dt+Cy/2 < abs(Yi[0][n-1])):
        Vu2 = max(abs(P[0]),abs(P[6]))
    elif (Xi[n-2][n-1]+Pile[3]/2<dt+Cx/2) and (abs(Yi[0][n-1]) <= dt+Cy/2 <= abs(Yi[0][n-1])+Pile[3]/2):
        Vu2 = max(abs(P[0]),abs(P[6]))/2
    elif (Xi[n-2][n-1]+Pile[3]/2<dt+Cx/2) and (abs(Yi[0][n-1])+Pile[3]/2<dt+Cy/2):
        Vu2 = 0
    else :
        Vu2 = 0
        
    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[1]),abs(P[0]+P[2]),abs(P[0]+P[3]),abs(P[1]+P[3]),abs(P[1]+P[4]),abs(P[2]+P[3]),abs(P[2]+P[5]),abs(P[3]+P[4]),abs(P[3]+P[5]),abs(P[3]+P[6]),abs(P[4]+P[6]),abs(P[5]+P[6]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산
        
    Vu4 = max(P[0],P[1],P[2],P[4],P[5],P[6])
    ΦVc4 = ΦVc4a(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(P[0],P[6])
    ΦVc5 = (Pi*vc/2*Lxy[0][n-1])*dt

    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(ΦVc5) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")

In [12]:
if n == 8:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-(sum(P)-P[2]-P[5])/2,d)))

    while True :
        
        if (d1/2+Cx/2 < Xi[n-1][n-1] and d1/2+Cy/2<Yi[2][n-1]) : 
            
            if ΦVc1(d1)<sum(P) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<(sum(P)-(P[2]+P[5])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) : 
            
            if ΦVc1(d1)<(sum(P)-(P[2]+P[5])) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) : 
            
            if ΦVc1(d1)<(sum(P)-(P[2]+P[5])) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2<Yi[2][n-1]) : 
            
            if ΦVc1(d1)<(sum(P)-(P[3]+P[4])/2) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5]+P[3]+P[4])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5])-(P[3]+P[4])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/4 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2<Yi[2][n-1]) : 
            
            if ΦVc1(d1)<sum[P]-(P[3]+P[4]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) : 
            
            if ΦVc1(d1)<sum[P]-(P[3]+P[4]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5])-(P[3]+P[4]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) : 
            
            if ΦVc1(d1)<sum(P)-(P[2]+P[5])-(P[3]+P[4])-(P[0]+P[1]+P[6]+P[7])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        if (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
            
            d1 = d1

        if (d1/2+Cx/2 < Xi[n-1][n-1] and d1/2+Cy/2<Yi[2][n-1] and ΦVc1(d1)>sum(P)) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc1(d1)>(sum(P)-(P[2]+P[5])/2)) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1] and ΦVc1(d1)>(sum(P)-(P[2]+P[5]))) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc1(d1)>(sum(P)-(P[2]+P[5]))) or \
           (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>(sum(P)-(P[2]+P[5]))) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2<Yi[2][n-1] and ΦVc1(d1)>(sum(P)-(P[3]+P[4])/2)) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc1(d1)>sum(P)-(P[2]+P[5]+P[3]+P[4])/2) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1] and ΦVc1(d1)>sum(P)-(P[2]+P[5])-(P[3]+P[4])/2) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc1(d1)>sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/4) or \
           (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2 and ΦVc1(d1)>sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2<Yi[2][n-1] and ΦVc1(d1)>sum[P]-(P[3]+P[4])) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc1(d1)>sum(P)-(P[2]+P[5])/2-(P[3]+P[4])) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1] and ΦVc1(d1)>sum(P)-(P[2]+P[5])-(P[3]+P[4])) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc1(d1)>sum(P)-(P[2]+P[5])-(P[3]+P[4])-(P[0]+P[1]+P[6]+P[7])/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2/(Pi*vc/2*Lxy[0][n-1]))
    d2b = Ceil(max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)

    while True :

        if (d2+Cx/2 < Xi[n-1][n-1] and d2+Cy/2<Yi[2][n-1]):

            if max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :
                L = Lxy[0][n-1]

                if d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :
                L = Lxy[0][n-1]

                if Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1]) :

            if max(abs(P[0]+P[1]),abs(P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :
                L = Lxy[0][n-1]

                if Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3]):

            if max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2 > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :
                L = Lxy[0][n-1]

                if Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if d2+Cx/2 < Xi[n-1][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (d2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]               

            if (d2+Cx/2 < abs(Xi[n-1][n-1])) and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])) :                         
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d2+Cy/2<Yi[2][n-1]):

            if max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :
                L = Lxy[0][n-1]

                if d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :
                L = Lxy[0][n-1]

                if Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1]) :

            if max(abs(P[0]+P[1]),abs(P[6]+P[7])) > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :
                L = Lxy[0][n-1]

                if Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7])) :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3]):

            if max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2 > max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :
                L = Lxy[0][n-1]

                if Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2 :
                    d2 = d2 + 0.05
                else :
                    d2 = d2

            else :
                L = Lxy[1][n-1]               

                if Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :                         
                    d2 = d2 + 0.05
                else : 
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]               

            if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L) < max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2 :                         
                d2 = d2 + 0.05
            else : 
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and d2+Cy/2<Yi[2][n-1]):

            L = Lxy[0][n-1]

            if d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2):

            L = Lxy[0][n-1]

            if Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L) < max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1]) :

            L = Lxy[0][n-1]

            if Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7])) :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3]):

            L = Lxy[0][n-1]

            if Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L) < max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2 :
                d2 = d2 + 0.05
            else :
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L = Lxy[1][n-1]
            d2 = d2

        if (d2+Cx/2 < Xi[n-1][n-1] and d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))) or \
           (d2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))) or \
           (d2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]),abs(P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))) or \
           (d2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2,max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))) or \
           (d2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]+Pile[3]/2<d2+Cy/2 and ΦVc2(d2,L)>max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))) or \
           (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)) or \
           (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)) or \
           (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]),abs(P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)) or \
           (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L)>max(max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2,max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)) or \
           (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2 and ΦVc2(d2,L)>max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and d2+Cy/2<Yi[2][n-1] and ΦVc2(d2,L)>max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7]))) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[2][n-1]<=d2+Cy/2<=Yi[2][n-1]+Pile[3]/2 and ΦVc2(d2,L)>max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7]))) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[2][n-1]+Pile[3]/2<d2+Cy/2<=Yi[0][n-1] and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[6]+P[7]))) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]<d2+Cy/2<=Yi[0][n-1]+Pile[3] and ΦVc2(d2,L)>max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[2]),abs(P[0]+P[3]),abs(P[1]+P[2]),abs(P[1]+P[4]),abs(P[2]+P[3]),abs(P[2]+P[4]),abs(P[2]+P[5]),abs(P[3]+P[5]),abs(P[3]+P[6]),abs(P[4]+P[5]),abs(P[4]+P[7]),abs(P[5]+P[6]),abs(P[5]+P[7])),d)))
    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile
        
    d4a = Ceil(max(solve(ΦVc4a(d)-max(P[3],P[4]),d)))
    d4b = Ceil(max(solve(ΦVc4b(d)-max(P[0],P[1],P[6],P[7]),d)))
    d4 = max(d4a,d4b)

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(P[0],P[1],P[6],P[7]),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if (d1/2+Cx/2 < Xi[n-1][n-1] and d1/2+Cy/2<Yi[2][n-1]) :
         Vu1 = sum(P)
    elif (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) :
         Vu1 = sum(P)-(P[2]+P[5])/2
    elif (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) :
         Vu1 = sum(P)-(P[2]+P[5])
    elif (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) :
         Vu1 = sum(P)-(P[2]+P[5])
    elif (d1/2+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
         Vu1 = sum(P)
    elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2<Yi[2][n-1]) :
         Vu1 = sum(P)-(P[3]+P[4])/2
    elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) : 
         Vu1 = sum(P)-(P[2]+P[5]+P[3]+P[4])/2
    elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) : 
         Vu1 = sum(P)-(P[2]+P[5])-(P[3]+P[4])/2
    elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) : 
         Vu1 = sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/4
    elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) :
         Vu1 = sum(P)-(P[2]+P[5])-(P[3]+P[4])/2-(P[0]+P[1]+P[6]+P[7])/2
    elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2<Yi[2][n-1]) :
         Vu1 = sum[P]-(P[3]+P[4])
    elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]<=d1/2+Cy/2<=Yi[2][n-1]+Pile[3]/2) :
         Vu1 = sum(P)-(P[2]+P[5])/2-(P[3]+P[4])
    elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[2][n-1]+Pile[3]/2<d1/2+Cy/2<=Yi[0][n-1]) :
         Vu1 = sum(P)-(P[2]+P[5])-(P[3]+P[4])
    elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]<d1/2+Cy/2<=Yi[0][n-1]+Pile[3]) :
         Vu1 = sum(P)-(P[2]+P[5])-(P[3]+P[4])-(P[0]+P[1]+P[6]+P[7])/2
    elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d1/2+Cy/2) : 
         Vu1 = 0
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < Xi[n-1][n-1] and dt+Cy/2<Yi[2][n-1]) :
        Vu2 = max(max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))
    elif (dt+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]<=dt+Cy/2<=Yi[2][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))
    elif (dt+Cx/2 < Xi[n-1][n-1] and Yi[2][n-1]+Pile[3]/2<dt+Cy/2<=Yi[0][n-1]) :
        Vu2 = max(max(abs(P[0]+P[1]),abs(P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))
    elif (dt+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]<dt+Cy/2<=Yi[0][n-1]+Pile[3]) :
        Vu2 = max(max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2,max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7])))
    elif (dt+Cx/2 < Xi[n-1][n-1] and Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and dt+Cy/2<Yi[2][n-1]) :
        Vu2 = max(max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]<=dt+Cy/2<=Yi[2][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[2][n-1]+Pile[3]/2<dt+Cy/2<=Yi[0][n-1]) :
        Vu2 = max(max(abs(P[0]+P[1]),abs(P[6]+P[7])),max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]<dt+Cy/2<=Yi[0][n-1]+Pile[3]) :
        Vu2 = max(max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2,max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]+P[6]),abs(P[1]+P[4]+P[7]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and dt+Cy/2<Yi[2][n-1]) :
        Vu2 = max(abs(P[0]+P[1]+P[2]),abs(P[5]+P[6]+P[7]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and Yi[2][n-1]<=dt+Cy/2<=Yi[2][n-1]+Pile[3]/2) :
        Vu2 = max(abs(P[0]+P[1]+P[2]/2),abs(P[5]/2+P[6]+P[7]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and Yi[2][n-1]+Pile[3]/2<dt+Cy/2<=Yi[0][n-1]) :
        Vu2 = max(abs(P[0]+P[1]),abs(P[6]+P[7]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and Yi[0][n-1]<dt+Cy/2<=Yi[0][n-1]+Pile[3]) :
        Vu2 = max(abs(P[0]+P[1]),abs(P[6]+P[7]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2 and Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = 0
    else :
        Vu2 = 0

    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[2]),abs(P[0]+P[3]),abs(P[1]+P[2]),abs(P[1]+P[4]),abs(P[2]+P[3]),abs(P[2]+P[4]),abs(P[2]+P[5]),abs(P[3]+P[5]),abs(P[3]+P[6]),abs(P[4]+P[5]),abs(P[4]+P[7]),abs(P[5]+P[6]),abs(P[5]+P[7]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산
        
    if d4a>d4b :
        Vu4 = max(P[3],P[4])
        ΦVc4 = ΦVc4a(dt)
    
    else :
        Vu4 = max(P[0],P[1],P[6],P[7])
        ΦVc4 = ΦVc4b(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(P[0],P[1],P[6],P[7])

    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")
    print(d1,d2,d3,d4,d5)

In [13]:
if n == 9:

 # 2 way shear at d/2 from column face

    d = Symbol('d')                         
    d1 = Ceil(max(solve(ΦVc1(d)-(sum(P)-P[4])/2,d)))

    while True :
        
        if (d1/2+Cx/2 < abs(Xi[n-1][n-1]) and d1/2+Cy/2 < abs(Yi[0][n-1])) :

            if ΦVc1(d1)<sum(P)-P[4] :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < abs(Xi[n-1][n-1]) and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :

            if ΦVc1(d1)<sum(P)-P[4]-(P[1]+P[7])/2 : 
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (d1/2+Cx/2 < abs(Xi[n-1][n-1]) and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :
        
            if ΦVc1(d1)<sum(P)-(P[1]+P[4]+P[7]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2 < abs(Yi[0][n-1])) :

            if ΦVc1(d1)<sum(P)-P[4]-(P[3]+P[5])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :
            
            if ΦVc1(d1)<3/4*(P[0]+P[2]+P[6]+P[8])+(P[1]+P[3]+P[5]+P[7])/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :
            
            if ΦVc1(d1)<(sum(P)-(P[1]+P[4]+P[7]))/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1
        
        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2 < abs(Yi[0][n-1])) :
        
            if ΦVc1(d1)<sum(P)-(P[3]+P[4]+P[5]) :
                d1 = d1 + 0.05
            else :
                d1 = d1

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) :
        
            if ΦVc1(d1)<(sum(P)-(P[3]+P[4]+P[5]))/2 :
                d1 = d1 + 0.05
            else :
                d1 = d1
        
        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) :
        
            d1 = d1
        

        if ((d1/2+Cx/2 < abs(Xi[n-1][n-1]) and d1/2+Cy/2 < abs(Yi[0][n-1])) and ΦVc1(d1)>sum(P)-P[4]) or \
           ((d1/2+Cx/2 < abs(Xi[n-1][n-1]) and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) and ΦVc1(d1)>sum(P)-P[4]-(P[1]+P[7])/2) or \
           ((d1/2+Cx/2 < abs(Xi[n-1][n-1]) and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) and ΦVc1(d1)>sum(P)-(P[1]+P[4]+P[7])) or \
           ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and d1/2+Cy/2 < abs(Yi[0][n-1])) and ΦVc1(d1)>sum(P)-P[4]-(P[3]+P[5])/2) or \
           ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) and ΦVc1(d1)>3/4*(P[0]+P[2]+P[6]+P[8])+(P[1]+P[3]+P[5]+P[7])/2) or \
           ((Xi[n-1][n-1]<=d1/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2) and ΦVc1(d1)>(sum(P)-(P[1]+P[4]+P[7]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and d1/2+Cy/2 < abs(Yi[0][n-1])) and ΦVc1(d1)>sum(P)-(P[3]+P[4]+P[5])) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])<=d1/2+Cy/2<=abs(Yi[0][n-1])+Pile[3]/2) and ΦVc1(d1)>(sum(P)-(P[3]+P[4]+P[5]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2 and abs(Yi[0][n-1])+Pile[3]/2<d1/2+Cy/2)) :
            break

 # 1 way shear at d from column face

    d2a = Ceil(max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2/(Pi*vc/2*Lxy[0][n-1]))                   
    d2b = Ceil(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2/(Pi*vc/2*Lxy[1][n-1]))
    d2 = min(d2a,d2b)
    
    while True :

        if (d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])):

            if max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))>max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])):
                L=Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L=Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2>max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])):
                L=Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L=Lxy[1][n-1]
            
                if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2):

            L=Lxy[1][n-1]
        
            if (d2+Cx/2 < abs(Xi[n-1][n-1]) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))):
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            if max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))>max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2:
                L=Lxy[0][n-1]

                if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])):
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L=Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2            

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            if max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2>max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2:
                L=Lxy[0][n-1]

                if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

            else:
                L=Lxy[1][n-1]
            
                if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2:
                    d2 = d2 + 0.05
                else:
                    d2 = d2

        elif (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :

            L=Lxy[1][n-1]
        
            if (Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2:
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d1/2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) :

            L=Lxy[0][n-1]

            if (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])):
                d2 = d2 + 0.05
            else:
                d2 = d2     

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2):

            L=Lxy[0][n-1]

            if (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L)<max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2:
                d2 = d2 + 0.05
            else:
                d2 = d2

        elif (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :

            L=Lxy[1][n-1]
            d2 = d2

        if ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])),max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])))) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])),max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2)) or \
           ((d2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L) > max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])))) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2)) or \
           ((Xi[n-1][n-1]<=d2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<d2+Cy/2) and ΦVc2(d2,L) > max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (d2+Cy/2 < abs(Yi[0][n-1])) and ΦVc2(d2,L) > max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))) or \
           ((Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2) and (Yi[0][n-1]<=d2+Cy/2<=Yi[0][n-1]+Pile[3]/2) and ΦVc2(d2,L) > max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2) or \
           (Xi[n-1][n-1]+Pile[3]/2<d2+Cx/2 and Yi[0][n-1]+Pile[3]/2<d2+Cy/2) :
            break

 # 2 Way Shear at d/2 from Pile (Internal)

    d3a = Ceil(max(solve(ΦVc3a(d)-max(list(map(abs,P))),d)))
    d3b = Ceil(max(solve(ΦVc3b(d)-max(abs(P[0]+P[1]),abs(P[0]+P[3]),abs(P[1]+P[2]),abs(P[1]+P[4]),abs(P[2]+P[5]),abs(P[3]+P[4]),abs(P[3]+P[6]),abs(P[4]+P[5]),abs(P[4]+P[7]),abs(P[5]+P[8]),abs(P[6]+P[7]),abs(P[7]+P[8])),d)))

    if d3a <= (S-Pile[3])/2:
        d3 = d3a
    
    else :
        d3 = d3b

 # 2 Way Shear at Corner Pile

    d4a = Ceil(max(solve(ΦVc4a(d)-max(abs(P[1]),abs(P[3]),abs(P[5]),abs(P[7])),d)))
    d4b = Ceil(max(solve(ΦVc4b(d)-max(abs(P[0]),abs(P[2]),abs(P[6]),abs(P[8])),d)))
    d4 = max(d4a,d4b)

 # 1 Way Shear at Corner Pile
    
    d5 = Ceil(max(solve(ΦVc5(d)-max(abs(P[0]),abs(P[2]),abs(P[6]),abs(P[8])),d)))

 # Design Check
    
    dt = max(d1,d2,d3,d4,d5)

    # 2 Way Shear at d/2 from Column face Vu 계산

    if (dt/2+Cx/2 < abs(Xi[n-1][n-1])) and (dt/2+Cy/2 < abs(Yi[0][n-1])) :
         Vu1 = sum(P)-P[4]
    elif (dt/2+Cx/2 < abs(Xi[n-1][n-1])) and (abs(Yi[0][n-1])<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
         Vu1 = sum(P)-P[4]-(P[1]+P[7])/2
    elif (dt/2+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2):
         Vu1 = sum(P)-(P[1]+P[4]+P[7])
    elif (Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt/2+Cy/2 < abs(Yi[0][n-1])) :
         Vu1 = sum(P)-P[4]-(P[3]+P[5])/2
    elif (Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
         Vu1 = 3/4*(P[0]+P[2]+P[6]+P[8])+(P[1]+P[3]+P[5]+P[7])/2
    elif (Xi[n-1][n-1]<=dt/2+Cx/2<=Xi[n-1][n-1]+Pile[3]/2 and Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2):
         Vu1 = (sum(P)-(P[1]+P[4]+P[7]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2) and (dt/2+Cy/2 < abs(Yi[0][n-1])):
         Vu1 = sum(P)-(P[3]+P[4]+P[5])
    elif (Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2) and (Yi[0][n-1]<=dt/2+Cy/2<=Yi[0][n-1]+Pile[3]/2):
         Vu1 = (sum(P)-(P[3]+P[4]+P[5]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt/2+Cx/2) and (Yi[0][n-1]+Pile[3]/2<dt/2+Cy/2):
         Vu1 = 0
    else :
         Vu1 = 0

    # 1 Way Shear at d from Column face Vu 계산

    if (dt+Cx/2 < abs(Xi[n-1][n-1])) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])),max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])))
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8])),max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2)
    elif (dt+Cx/2 < abs(Xi[n-1][n-1])) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8])))
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2,max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2)
    elif (Xi[n-1][n-1]<=dt+Cx/2<=Xi[n-1][n-1]+Pile[3]/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = max(abs(P[0]+P[3]+P[6]),abs(P[2]+P[5]+P[8]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (dt+Cy/2 < abs(Yi[0][n-1])) :
        Vu2 = max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]<=dt+Cy/2<=Yi[0][n-1]+Pile[3]/2) :
        Vu2 = max(abs(P[0]+P[1]+P[2]),abs(P[6]+P[7]+P[8]))/2
    elif (Xi[n-1][n-1]+Pile[3]/2<dt+Cx/2) and (Yi[0][n-1]+Pile[3]/2<dt+Cy/2) :
        Vu2 = 0
    else :
        Vu2 = 0

    # 2 Way Shear at d/2 from Pile Vu 계산
    
    if dt <= (S-Pile[3])/2 :
        Vu3 = max(list(map(abs,P)))
        ΦVc3 = ΦVc3a(dt)
    else :
        Vu3 = max(abs(P[0]+P[1]),abs(P[0]+P[3]),abs(P[1]+P[2]),abs(P[1]+P[4]),abs(P[2]+P[5]),abs(P[3]+P[4]),abs(P[3]+P[6]),abs(P[4]+P[5]),abs(P[4]+P[7]),abs(P[5]+P[8]),abs(P[6]+P[7]),abs(P[7]+P[8]))
        ΦVc3 = ΦVc3b(dt)

    # 2 Way Shear at Corner Pile Vu 계산

    if d4a>=d4b : 
        Vu4 = max(abs(P[1]),abs(P[3]),abs(P[5]),abs(P[7]))
        ΦVc4 = ΦVc4a(dt)
    else :
        Vu4 = max(abs(P[0]),abs(P[2]),abs(P[6]),abs(P[8]))
        ΦVc4 = ΦVc4b(dt)

    # 1 Way Shear at Corner Pile Vu 계산

    Vu5 = max(abs(P[0]),abs(P[2]),abs(P[6]),abs(P[8]))

    print("d = " + str(dt) + " (Governing Thickness)")
    print("2 Way Shear at d/2 From Column Face\n" + "    ΦVc = " + str(round(ΦVc1(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu1,3)) + " kN")
    print("1 Way Shear at d From Column Face\n" + "    ΦVc = " + str(round(ΦVc2(dt,L),3)) + " kN\n" + "    Vu = " + str(round(Vu2,3)) + " kN")
    print("2 Way Shear at d/2 from Pile\n" + "    ΦVc = " + str(round(ΦVc3,3)) + " kN\n" + "    Vu = " + str(round(Vu3,3)) + " kN")
    print("2 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc4,3)) + " kN\n" + "    Vu = " + str(round(Vu4,3)) + " kN")
    print("1 Way Shear at Corner Pile\n" + "    ΦVc = " + str(round(ΦVc5(dt),3)) + " kN\n" + "    Vu = " + str(round(Vu5,3)) + " kN")

d = 0.6500000000000001 (Governing Thickness)
2 Way Shear at d/2 From Column Face
    ΦVc = 5674.488 kN
    Vu = 3598.222 kN
1 Way Shear at d From Column Face
    ΦVc = 1063.966 kN
    Vu = 674.667 kN
2 Way Shear at d/2 from Pile
    ΦVc = 3770.786 kN
    Vu = 899.556 kN
2 Way Shear at Corner Pile
    ΦVc = 1297.352 kN
    Vu = 449.778 kN
1 Way Shear at Corner Pile
    ΦVc = 875.917 kN
    Vu = 449.778 kN


In [14]:
Depth = dt+Pile[5]
Conc = Lxy[0][n-1]*Lxy[1][n-1]*Depth

print(round(Depth,3))
print(round(Conc,3))

0.75
5.467
